# DN4 - prvi del

In [12]:
import numpy as np
import pandas as pd

from eq_regression import *

# run olny the fist time to create extedent dataframes
#%run preproces.py


def XY(data):
    X = data.drop('Q', axis=1)
    y = data['Q']
    return X, y

Najprej sem nalozil vse podatke in jih spravi v primerno obliko za regresijo.

In [13]:

data = pd.read_csv('DN4_1_podatki.csv')
X, y = XY(data)

df1 = pd.read_csv('NEW_COLL_DATA_1.csv')
X1, _ = XY(df1)
df11 = pd.read_csv('NEW_COLL_DATA_ALL1.csv')
X11, _ = XY(df11)

df2 = pd.read_csv('NEW_COLL_DATA_2.csv')
X2, _ = XY(df2)
df22 = pd.read_csv('NEW_COLL_DATA_ALL2.csv')
X22, _ = XY(df22)

df3 = pd.read_csv('NEW_COLL_DATA_3.csv')
X3, _ = XY(df3)
df33 = pd.read_csv('NEW_COLL_DATA_ALL3.csv')
X33, _ = XY(df33)

df4 = pd.read_csv('NEW_COLL_DATA_4.csv')
X4, _ = XY(df4)
df44 = pd.read_csv('NEW_COLL_DATA_ALL4.csv')
X44, _ = XY(df44)

DF = [X, X1, X11, X2, X22, X3, X33, X4, X44]

Testeranje regresije in primerjava rezultatov

## Linearna regresija

In [14]:
for el in DF:
    best = np.infty
    for k in range(1, 10):
        meja = 10**(-k)
        lr, err = linearna_regresija(el, y, meja=meja)
        if err < best:
            best = err
    print(f"Best error: {best}")

Best error: 0.4586326636044633
Best error: 0.029066207515417067
Best error: 0.02906564662164767
Best error: 0.029066207515417067
Best error: 0.02906564662164767
Best error: 0.029066207515417067
Best error: 0.02906564662164767
Best error: 0.029066207515417067
Best error: 0.02906564662164767


## Ridge regresija

In [15]:
for el in DF:
    best = np.infty
    for k in range(1, 10):
        meja = 10**(-k)
        lr, err = ridge_regresija(el, y, lam=1,  meja=meja)
        if err < best:
            best = err
    print(f"Best error: {best}")

Best error: 0.4586622367403794
Best error: 0.03066417688551802
Best error: 0.030600525943871255
Best error: 0.03066417688551802
Best error: 0.030600525943871255
Best error: 0.03066417688551802
Best error: 0.030600525943871255
Best error: 0.03066417688551802
Best error: 0.030600525943871255


## Lasso regresija

In [16]:
for el in DF:
    best = np.infty
    for k in range(1, 10):
        meja = 10**(-k)
        lr, err = lasso_regresija(el, y, lam=1,  meja=meja)
        if err < best:
            best = err
    print(f"Best error: {best}")

Best error: 0.4586432706757743
Best error: 0.04211915288233077
Best error: 0.04314170050850309
Best error: 0.030817470291243716
Best error: 0.038539156114306604
Best error: 0.031581304333365035
Best error: 0.031089573092939277
Best error: 0.03265165194577597
Best error: 0.0360644246758238


Ugotovitev: najboljsi model je z linearno regresijo stem da generiramo nove spremenljivke
Napaka je $0.029$

## Drugi pristopi

In [17]:
from pysr import PySRRegressor
import sympy 

np.random.seed(1234)

model = PySRRegressor(
    niterations=1000,  # < Increase me for better results
    binary_operators=["+", "*", "-", "/"],
    unary_operators=[
        "cos",
        "sin",
        "inv(x) = 1/x",
        "square",
        "sqrt",
        "cube",
        "cos2(x)=cos(x)^2",
        "sin2(x)=sin(x)^2"
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x,
                          "cos2": lambda x: sympy.cos(x)**2,
                          "sin2": lambda x: sympy.sin(x)**2},
    # ^ Define operator for SymPy as well
    maxsize=20,
    # ^ Allow greater complexity.
    maxdepth=10,
    # ^ But, avoid deep nesting.
    progress=False,
    # ^ Can set to false if printing to a file.
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
    timeout_in_seconds=60 * 10,
    
    warm_start=True
)

In [18]:
model.fit(X, y)

/Users/vitorozman/Documents/School/Master/1.Letnik/ML/Advanced_ML/venv/lib/python3.11/site-packages/pysr/sr.py:1296: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Expressions evaluated per second: 2.580e+05
Head worker occupation: 10.3%
Progress: 568 / 15000 total iterations (3.787%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.150e+00  3.109e-07  0.6829838
2           1.026e+00  1.144e-01  sin(theta)
4           9.466e-01  4.011e-02  cos2(Ta / Tw)
5           6.004e-01  4.552e-01  ((Tw - 21.409678) / Ta)
7           5.509e-01  4.303e-02  ((cube(Tw) / square(Ta)) / 243.64381)
8           4.368e-01  2.320e-01  (((Tw * sin(theta)) / Ta) - eta)
9           3.286e-01  2.846e-01  square((Tw * sin(theta)) / (Ta - -11.573738))
10          2.969e-01  1.017e-01  square((Tw * sqrt(sin(theta))) / (Ta - -11.573738))
11          2.843e-01  4.330e-02  ((square(Tw / (Ta - -6.1759443)) - 0.9991415) * sin(theta))
12          1.438e-01  6.814e-01  (square((Tw / (Ta - -6.1640897)) - eta) / inv(sin(theta)))
14          1.035e-01  1.

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                          0.6829838   
	1         0.114405                                         sin(theta)   
	2         0.015705                                 (0.019037813 * Tw)   
	3         0.064508                                      cos2(Ta / Tw)   
	4         0.504256                            ((Tw - Ta) / 17.736282)   
	5         0.285202                       square((Tw - Ta) / 21.88182)   
	6         0.153915              (((Tw - Ta) / 11.41495) * sin(theta))   
	7         0.926354      (square((Tw - Ta) / -17.496422) * sin(theta))   
	8         0.400656  (square(((Tw - Ta) / -14.056076) + eta) * sin(...   
	9         1.074180  (square((Tw - Ta) / (-15.158528 / cos(eta))) *...   
	10        0.119789  (square((Tw - Ta) / (-13.868196 / cos(sqrt(eta...   
	11  >>>>  0.211561  ((square((Tw - Ta) / 13.056252) * sin(theta)) ...   
	12        0.058718  ((square((Tw - Ta) / -13.821489) * sin(theta))...   
	13        0.000323  (square((Tw - Ta) / (14.877709 / cos2(sin(sin(...   
	14        0.021434  (square((Tw - Ta) / (cube(2.0897014 * sqrt(eta...   
	15        0.000010  (square((Tw - Ta) / (cube(2.0897014 * sqrt(eta...   
	16        0.000113  (square(((Tw + 0.022311283) - Ta) / (cube(2.08...   
	17        0.012354  (square((Tw - Ta) / (cube(2.0707357 * sqrt(eta...   
	
	        loss  complexity  
	0   1.149930           1  
	1   1.025619           2  
	2   1.009637           3  
	3   0.946564           4  
	4   0.571682           5  
	5   0.429826           6  
	6   0.315940           8  
	7   0.125110           9  
	8   0.056142          11  
	9   0.019177          12  
	10  0.017012          13  
	11  0.013768          14  
	12  0.012983          15  
	13  0.012979          16  
	14  0.012704          17  
	15  0.012703          18  
	16  0.012702          19  
	17  0.012546          20  
]

In [19]:
print(model.equations_)

    complexity      loss     score  \
0            1  1.149930  0.000000   
1            2  1.025619  0.114405   
2            3  1.009637  0.015705   
3            4  0.946564  0.064508   
4            5  0.571682  0.504256   
5            6  0.429826  0.285202   
6            8  0.315940  0.153915   
7            9  0.125110  0.926354   
8           11  0.056142  0.400656   
9           12  0.019177  1.074180   
10          13  0.017012  0.119789   
11          14  0.013768  0.211561   
12          15  0.012983  0.058718   
13          16  0.012979  0.000323   
14          17  0.012704  0.021434   
15          18  0.012703  0.000010   
16          19  0.012702  0.000113   
17          20  0.012546  0.012354   

                                             equation  \
0                                           0.6829838   
1                                          sin(theta)   
2                                  (0.019037813 * Tw)   
3                                       cos2(Ta /